In [56]:
import pandas as pd
df = pd.read_csv("moviescombinedGenerid.csv")
df.head()

,youtubeId,movieId,title,genres,youtube_link,genre_ids,transcription
0,K26_sDKnvMU,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,https://www.youtube.com/watch?v=K26_sDKnvMU,"[1, 2, 3, 8]",sergeant yes sir establish your recon post dow...
1,3LPANjHlPxo,2,Jumanji (1995),Adventure|Children|Fantasy,https://www.youtube.com/watch?v=3LPANjHlPxo,"[1, 8]",I'm glad you decided to buy this place I'm sur...
2,rEnOoWs3FuA,3,Grumpier Old Men (1995),Comedy|Romance,https://www.youtube.com/watch?v=rEnOoWs3FuA,"[3, 13]",NaN
3,j9xml1CxgXI,4,Waiting to Exhale (1995),Comedy|Drama|Romance,https://www.youtube.com/watch?v=j9xml1CxgXI,"[3, 6, 13]",NaN
4,ltwvKLnj1B4,5,Father of the Bride Part II (1995),Comedy,https://www.youtube.com/watch?v=ltwvKLnj1B4,[3],NaN


In [57]:
df = df.dropna(subset=['transcription'])

In [58]:
df['genres'] = df['genres'].apply(lambda x: x.split('|'))

In [59]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Sample text
text = "The quick brown fox jumps over the lazy dog. Dogs are friendly animals."
def preprocess_text(text):
  tokens = word_tokenize(text)
  tokens = [word.lower() for word in tokens if word not in string.punctuation]
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]
  stemmer = PorterStemmer()
  stemmed_tokens = [stemmer.stem(word) for word in tokens]
  return " ".join(stemmed_tokens)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
df["processed_transcription"]= df["transcription"].apply(preprocess_text)

In [61]:

columns_to_drop = ['youtubeId', 'movieId', 'title', 'genre_ids', 'youtube_link','transcription']
df = df.drop(columns=columns_to_drop)

df

,genres,processed_transcription
0,"[Adventure, Animation, Children, Comedy, Fantasy]",sergeant ye sir establish recon post downstair...
1,"[Adventure, Children, Fantasy]",'m glad decid buy place 'm sure kid happi ever...
5,"[Action, Crime, Thriller]",music 's feel search scent prey hunt 's thing ...
7,"[Adventure, Children]",think 're go 'm run away home jonathan taylor ...
9,"[Action, Adventure, Thriller]",world target 72 hour ago secret weapon system ...
...,...,...
19560,[Documentary],'s nobodi sign 's rope 's patrolman 's nobodi ...
19561,"[Comedy, Documentary, War]",oh welcom beauti afghanistan safeti spoon case...
19572,[Comedy],music paramount pictur invit meet man search l...
19577,[Thriller],name 's robert n't smoke n't pet see come toda...


In [62]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
X = df["processed_transcription"]
mlb = MultiLabelBinarizer()

# Fit and transform labels
y = mlb.fit_transform(df['genres'])

vectorizer = TfidfVectorizer(max_features=2500, max_df=0.9)
vectorizer.fit(X)

TfidfVectorizer(max_df=0.9, max_features=2500)

In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [64]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

clf = MultiOutputClassifier(LogisticRegression()).fit(X_train_tfidf, y_train)

In [65]:
prediction = clf.predict(X_test_tfidf)
prediction

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [66]:
from sklearn.metrics import accuracy_score
print('Accuracy Score: ', accuracy_score(y_test, prediction))

Accuracy Score:  0.1326530612244898


In [69]:
#save model
import pickle
pickle.dump(clf, open('model.pkl', 'wb'))

#save vectorizer
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))

#save multi laber bineryizer
pickle.dump(mlb, open('mlb.pkl', 'wb'))